## Pre work 

setup environment. Supports different env for Google colab or local env.


### combine to MP4 video file

using 12 framerate.

`ffmpeg -framerate 12 -i 000%03d.jpg output.mp4`

#### File locates bakcup:

[https://lxfy.keybase.pub/9517.mp4](https://lxfy.keybase.pub/9517.mp4)

### colab env

In [0]:
# Mount google drive to read images from colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### load dataset in colab env

All files should located in the dir: `/content/drive/My Drive/DevEnv/9517/Group_Component/sequence/000795.jpg`

In [0]:
# %cd /content/drive/My Drive/DevEnv
# %mkdir 9517/
# !unzip Group_Component.zip -d '/content/drive/My Drive/DevEnv/9517/'
# %cd /content/drive/My Drive/DevEnv/9517/Group_Component/
# !wget https://lxfy.keybase.pub/9517.mp4

In [0]:
VIDEO_PATH = '/content/drive/My Drive/DevEnv/9517/Group_Component/9517.mp4'
OUTPUT_PATH = '/content/drive/My Drive/DevEnv/9517/Group_Component/output/'

### local env

In [0]:
# # local dev env
# VIDEO_PATH = '/home/gao/Downloads/Group_Component/sequence/9517.mp4'
# FRAME_PATH = '/home/gao/Downloads/Group_Component/sequence/'
# OUTPUT_PATH = '/home/gao/Downloads/Group_Component/output/'

# Proj code

project code starts here.

In [0]:
import numpy as np
import cv2
from imutils.object_detection import non_max_suppression

In [0]:
# constrain define

# threshold of pedestrain border
COUNTER_THRESHOLD = 170

# read format
READ_FORMAT = 'video'

# crop image or not
IMG_CROP = False

# Frames per second
FPS = 12.0

OUTPUT_NAME = 'output2.avi'

OUTPUT_FULL_PATH = OUTPUT_PATH + OUTPUT_NAME

In [0]:
# change to save dir
%cd $OUTPUT_PATH

/content/drive/My Drive/DevEnv/9517/Group_Component/output


#### Read and write video

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html

https://docs.opencv.org/master/dd/d9e/classcv_1_1VideoWriter.html#afec93f94dc6c0b3e28f4dd153bc5a7f0

In [0]:
# read in as a video clip
cap = cv2.VideoCapture(VIDEO_PATH)

# Try different video encoder
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# fourcc= cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter(OUTPUT_NAME,fourcc, FPS, (768,576), True)


In [0]:
# define background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(
    history=500, varThreshold=100, detectShadows=False)

# using SVM+HOG
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())


## Utils

* https://www.zhihu.com/question/39405815

In [0]:
def compute_iou(box1, box2, wh=False):
    """
    compute the iou of two boxes.
    Args:
            box1, box2: [xmin, ymin, xmax, ymax] (wh=False) or [xcenter, ycenter, w, h] (wh=True)
            wh: the format of coordinate.
    Return:
            iou: iou of box1 and box2.
    """
    if wh == False:
            xmin1, ymin1, xmax1, ymax1 = box1
            xmin2, ymin2, xmax2, ymax2 = box2
    else:
            xmin1, ymin1 = int(box1[0]-box1[2]/2.0), int(box1[1]-box1[3]/2.0)
            xmax1, ymax1 = int(box1[0]+box1[2]/2.0), int(box1[1]+box1[3]/2.0)
            xmin2, ymin2 = int(box2[0]-box2[2]/2.0), int(box2[1]-box2[3]/2.0)
            xmax2, ymax2 = int(box2[0]+box2[2]/2.0), int(box2[1]+box2[3]/2.0)

    # get coord of two boxes
    xx1 = np.max([xmin1, xmin2])
    yy1 = np.max([ymin1, ymin2])
    xx2 = np.min([xmax1, xmax2])
    yy2 = np.min([ymax1, ymax2])

    # calc box area
    area1 = (xmax1-xmin1) * (ymax1-ymin1) 
    area2 = (xmax2-xmin2) * (ymax2-ymin2)

    inter_area = (np.max([0, xx2-xx1])) * (np.max([0, yy2-yy1])) 
    iou = inter_area / (area1+area2-inter_area+1e-6)
    return iou

## Method 1 

Direct detect objects

In [0]:
def getPerson(image):

    # get the front mask
    mask = fgbg.apply(frame)

    # cancelling the noise
    line = cv2.getStructuringElement(cv2.MORPH_CROSS, (1, 5), (-1, -1))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, line)
    # cv2.imshow("mask", mask)

    # find the max area contours
    out, contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in range(len(contours)):
        area = cv2.contourArea(contours[c])
        if area < COUNTER_THRESHOLD:
            continue
        rect = cv2.minAreaRect(contours[c])
        cv2.ellipse(image, rect, (0, 255, 0), 2, 8)
        cv2.circle(image, (np.int32(rect[0][0]), np.int32(rect[0][1])), 2, (255, 0, 0), 2, 8, 0)
    return image, mask

In [0]:
# while True:
#     ret, frame = cap.read()
#     result, m_ = getPerson(frame)
#     cv2.imshow('result', result)
#     k = cv2.waitKey(50)&0xff
#     if k == 27:
#         cv2.imwrite("result.png", result)
#         cv2.imwrite("mask.png", m_)

#         break
# cap.release()
# cv2.destroyAllWindows()

## Method 2

* https://blog.csdn.net/leo_10/article/details/89300437

* https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html

* [非极大值抑制](https://www.zhihu.com/question/39405815)

In [0]:
# calculate HOG position
def getHOGList(imageList):
    HOGList = []
    hog = cv2.HOGDescriptor()
    for i in range(len(imageList)):
        gray = cv2.cvtColor(imageList[i], cv2.COLOR_BGR2GRAY)
        HOGList.append(hog.compute(gray))
    return HOGList

In [0]:
# Get detector
def getHOGDetector(svm):
    sv = svm.getSupportVectors()
    rho, _, _ = svm.getDecisionFunction(0)
    sv = np.transpose(sv)
    return np.append(sv, [[-rho]], 0)

In [0]:
#get hard example
def getHardExamples(negImageList, svm):
    hardNegList = []
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(getHOGDetector(svm))
    for i in range(len(negImageList)):
        rects, wei = hog.detectMultiScale(negImageList[i], winStride=(4, 4),padding=(8, 8), scale=1.05)
        for (x,y,w,h) in rects:
            hardExample = negImageList[i][y:y+h, x:x+w]
            hardNegList.append(cv2.resize(hardExample,(64,128)))
    return hardNegList

In [0]:
# NonMaxSuppression
def fastNonMaxSuppression(boxes, sc, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # initialize the list of picked indexes
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    scores = sc
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the score of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(scores)
 
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
 
        idxs = np.delete(idxs, np.concatenate(([last],
                                               np.where(overlap > overlapThresh)[0])))
 
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick]

In [0]:
# Customize SVM

svm = cv2.ml.SVM_create()
svm.setCoef0(0.0)
svm.setDegree(3)
criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 1000, 1e-3)#End limit
svm.setTermCriteria(criteria)
svm.setGamma(0)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setNu(0.5)
svm.setP(0.1)  # for EPSILON_SVR, epsilon in loss function?
svm.setC(0.01)  # From paper, soft classifier 
svm.setType(cv2.ml.SVM_EPS_SVR)  # C_SVC # EPSILON_SVR # may be also NU_SVR # do regression task
# svm.train(np.array(hosList), cv2.ml.ROW_SAMPLE, np.array(labels))

In [0]:
def getPerson(image):
    # detect person using HOG
    (rects, weights) = hog.detectMultiScale(image, winStride=(4, 4),
                                            padding=(8, 8), scale=1.05)
    print(hog.detectMultiScale(image, winStride=(4, 4),
                                padding=(8, 8), scale=1.05))

    # draw border
    for (x, y, w, h) in rects:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # applyNon-Maximum Suppression
    # try a BIG threshold
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

    # draw final border canvas
    for (xA, yA, xB, yB) in pick:
        cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)
    # cv2.circle(image, (np.int32(rect[0][0]), np.int32(rect[0][1])), 2, (255, 0, 0), 2, 8, 0)
    return image

In [0]:


while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        result = getPerson(frame)

        # write the flipped frame
        out.write(frame)
        # cv2.imshow('frame',frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()


流式输出内容被截断，只能显示最后 5000 行内容。
(array([[407, 121,  96, 192],
       [349, 154,  67, 133]], dtype=int32), array([[1.00988607],
       [0.92290099]]))
(array([[349, 158,  66, 131]], dtype=int32), array([[1.13613528]]))
(array([[440, 179,  67, 134],
       [350, 160,  66, 131],
       [601,  59,  89, 178]], dtype=int32), array([[0.41027156],
       [1.35113487],
       [0.70327087]]))
(array([[352, 159,  66, 133],
       [446, 181,  68, 136],
       [157,   0,  65, 123],
       [602,  63,  86, 172]], dtype=int32), array([[1.32890239],
       [1.44327115],
       [0.2569122 ],
       [0.82877325]]))
(array([[352, 159,  66, 132],
       [452, 188,  68, 136],
       [603,  61,  86, 172]], dtype=int32), array([[1.47860501],
       [1.33186458],
       [0.78800508]]))
(array([[352, 157,  67, 134],
       [699, 189,  69, 145],
       [149,   0,  67, 127],
       [467, 188,  65, 130],
       [537,  36,  97, 194]], dtype=int32), array([[1.7534605 ],
       [2.37133045],
       [0.57209973],
       [1

In [0]:
%pwd

'/content/drive/My Drive/DevEnv/9517/Group_Component/output'